### Imports

In [3]:
import pandas as pd

### Input For Manipulation

In [7]:
folder_name = "BULK-StrictEqualityCatRestriction3/" # This must end in a slash
multi_obj = False

ages = [2, 4, 9, 14, 19, 31, 51]
genders = ["Male", "Female"]
objectives = []
foods = []
for age in ages:
    for gender in genders:
        out_name = f"{gender}-{age}"
        outname = "./ModelOutput/" + folder_name + out_name + ".xlsx" 
        ob = pd.read_excel(outname, sheet_name="Rankings")
        ob.loc[:, "Age"] = age
        ob.loc[:, "Gender"] = gender
        objectives.append(ob)
        
        fd = pd.read_excel(outname, sheet_name="Foods")
        fd.loc[:, "Age"] = age
        fd.loc[:, "Gender"] = gender
        foods.append(fd)
ob_bunch = pd.concat(objectives).rename({"Unnamed: 0": "Restaurant"}, axis=1)
fd_bunch = pd.concat(foods).rename({"Unnamed: 0": "Restaurant"}, axis=1)

In [8]:
pd.pivot_table(fd_bunch[fd_bunch["Restaurant"] == "Blimpie"].loc[:, ["Food", "Objective", "Amount", "Gender", "Age"]], index=["Food", "Objective"], columns=["Gender", "Age"], observed=True)

Amount                      \
Gender                                             Female                       
Age                                                    2         4         9    
Food                          Objective                                         
Bagel                         Calories From Fat  0.594633  0.819697       NaN   
                              Sodium                  NaN       NaN       NaN   
                              Sugars                  NaN       NaN       NaN   
Bluffin, Plain                Calories From Fat       NaN  0.066667  3.000000   
                              Sodium                  NaN       NaN  1.374657   
                              Sugars                  NaN       NaN  0.340005   
Chicken Caesar                Sodium                  NaN       NaN       NaN   
Garden                        Calories From Fat  2.517891  3.000000  3.000000   
                              Sodium             3.000000  3.000000  3.000000   
                              Sugars                  NaN       NaN  2.034786   
Grande Chili with Bean & Beef Calories From Fat       NaN       NaN  0.022274   
                              Sodium                  NaN       NaN       NaN   
                              Sugars             0.404486  0.501937  0.238611   
Mustard, Yellow Deli Style    Sugars                  NaN       NaN       NaN   
Popcorn, large                Calories From Fat       NaN       NaN       NaN   
                              Sodium             0.039270  0.070297  0.112360   
                              Sugars             0.022838  0.027606  0.096031   
Pretzels Classic Thin Style   Calories From Fat       NaN       NaN  0.150774   
                              Sodium             2.280381  1.662411  0.996504   
                              Sugars             0.902089  0.907344       NaN   
Red Wine Vinegar              Calories From Fat  3.000000  3.000000  3.000000   
                              Sodium             3.000000  3.000000  3.000000   
                              Sugars             3.000000  3.000000  3.000000   
Soft Pretzel, cinnamon-sugar  Calories From Fat  0.627589  0.624747  2.027224   
                              Sodium             0.680349  1.267292  1.891136   
                              Sugars                  NaN       NaN       NaN   
Soft Pretzel, plain           Calories From Fat  0.926554  1.177778  0.071949   
                              Sodium                  NaN       NaN       NaN   
                              Sugars             1.463267  1.838499  2.903969   

                                                                               \
Gender                                                                          
Age                                                    14        19        31   
Food                          Objective                                         
Bagel                         Calories From Fat  0.486822       NaN  0.486822   
                              Sodium                  NaN       NaN       NaN   
                              Sugars                  NaN  0.195079       NaN   
Bluffin, Plain                Calories From Fat  2.513178  3.000000  2.513178   
                              Sodium             2.735456  3.000000  2.735456   
                              Sugars             1.815922  2.804921  1.815922   
Chicken Caesar                Sodium                  NaN       NaN       NaN   
Garden                        Calories From Fat  3.000000  3.000000  3.000000   
                              Sodium             3.000000  3.000000  3.000000   
                              Sugars             1.306922  3.000000  1.306922   
Grande Chili with Bean & Beef Calories From Fat  0.127907  0.213660  0.127907   
                              Sodium                  NaN  0.097008       NaN   
                              Sugars             0.365128  0.057696  0.365128   
